In [1]:
import scanpy as sc

/Users/chrislangseth/miniforge3/envs/cellcharter/lib/python3.11/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [2]:
adata = sc.read_h5ad('/Users/chrislangseth/Downloads/GSM7990097_e13_mouse_head.h5ad')

ValueError: Layer 'raw' needs to be of one of <class 'numpy.ndarray'>, <class 'numpy.ma.MaskedArray'>, <class 'scipy.sparse._csr.csr_matrix'>, <class 'scipy.sparse._csc.csc_matrix'>, <class 'scipy.sparse._csr.csr_array'>, <class 'scipy.sparse._csc.csc_array'>, <class 'h5py._hl.dataset.Dataset'>, <class 'zarr.core.array.Array'>, <class 'anndata.compat.ZappyArray'>, <class 'anndata.abc.CSRDataset'>, <class 'anndata.abc.CSCDataset'>, <class 'dask.array.core.Array'>, <class 'anndata.compat.CupyArray'>, <class 'anndata.compat.CupySparseMatrix'>, <class 'anndata.compat.AwkArray'>, or <class 'xarray.core.dataarray.DataArray'>, not <class 'anndata._core.anndata.AnnData'>.

In [ ]:
adata